In [2]:
#ROB538 HW3

#Problem 1

import numpy as np
import matplotlib.pyplot as plt


class Agent:
    def __init__(self,K):
        self.g = 0
        self.states = []  # record position and action taken at the position
        self.actions = list(range(0,K)) #Set list of days as actions
        self.lr = 0.2 #learning rate, takes longer when low
        self.exp_rate = 0.01 #exploration rate .7
        self.decay_gamma = 0.85 #Reward decay does not work well when 1 .85

        self.switch = False

        # initial Q values
        self.Q_values = list(range(0,K))
        for a in self.actions:
            self.Q_values[a] = 0  # Q value is a dict of dict

    def giveReward(self,x_k,b,x_k_z):
        self.indiv = x_k*np.exp(-x_k/b)
        
        self.G = 0 
        for x_k_g in x_k_z:
            self.G = x_k_g*np.exp(-x_k_g/b) +self.G
        #Add a difference reward here later
        self.minusi = 0
        self.diff = 0
        #Get list for if agent didnt exist
        self.x_k_2 = np.array(x_k_z)
        self.x_k_2[self.action] = self.x_k_2[self.action] - 1
        #Repeat reward calculation
        for x_k_g2 in self.x_k_2:
            self.minusi = x_k_g2*np.exp(-x_k_g2/b) + self.minusi
        #Calculate difference reward
        self.diff = self.G-self.minusi
        #Now calculate "average difference reward"
        days = len(self.actions)
        self.x_k_2 = self.x_k_2 + 1/days
        for x_k_g2 in self.x_k_2:
            self.minusi = x_k_g2*np.exp(-x_k_g2/b) + self.minusi
        self.diff2 = self.G-self.minusi
        

        return self.indiv,self.G,self.diff,self.diff2

    def State(self):
        pass

    def chooseAction(self):
        # choose action with most expected value
        mx_nxt_reward = 0
        action = ""

        #Exploration action
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            for a in self.actions:
                nxt_reward = self.Q_values[a]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
                
        if action == "":    
            action = np.random.choice(self.actions)
            # print("current pos: {}, greedy aciton: {}".format(self.State.state, action))
        return action
    
    def reset(self):
        self.states = []
        self.reward = 0
        self.switch = False
    
    
    def play(self, i,x_k,b,x_k_z,rtype):
        if not self.switch:
            # to the end of game back propagate reward
            self.action = self.chooseAction()
            # append trace
            self.states.append(self.action) #Choose 1-6
            #print("current position {} action {}".format(self.State.state, action))
            # mark is end
            #self.State.isEndFunc()
            self.switch = True
            return(i,self)
        else:
        # back propagate
            reward1,reward2,reward3,reward4 = self.giveReward(x_k,b,x_k_z)
            rlist = [reward1,reward2,reward3,reward4]
            reward = rlist[rtype]
            self.Q_values[self.action] = reward
            for s in reversed(self.states):
                current_q_value = self.Q_values
                reward = current_q_value + self.lr * (self.decay_gamma * reward - current_q_value)
                self.Q_values = reward
            self.reset()
            i += 1
            return(i,self)
        



if __name__ == "__main__":
    def run(N,b,k,cycles,reward_type):
        #initialize agents
        #N = 25
        Agent_list = []
        Action_list = [0]*N
        
        #b = 5
        #k = 7
        x_k = [0]*N
        Count_list = [0]*k
        Decision = [0]*k
        #Create list of agent objects, each initialized
        for i in range(N):
            Agent_list.append(Agent(k))

        #print(Agent_list[8].g) gives g
        #Agent_list[8].indiv_reward(8,5)
        #print(Agent_list[8].indiv)
        i = np.zeros(N)
        count = 0
        max = 0
        while count<1:
            i = np.zeros(N)
            while i[0]<cycles:
                for x in range(N):
                    i[x],Agent_list[x] = Agent_list[x].play(i[x],x_k[x],b,Count_list,reward_type)
                    #Get info on actions here
                    Action_list[x] = Agent_list[x].action
                    #Count up all
                    Count_list[Action_list[x]] = Count_list[Action_list[x]] + 1
                for y in range(N):
                    #Set all x_k values and run again 
                    x_k[y] = Count_list[Action_list[y]] #Number of people who attended same day
                for z in range(N):
                    #Now distribute rewards
                    i[z],Agent_list[z] = Agent_list[z].play(i[z],x_k[z],b,Count_list,reward_type)
                #Reset Count List?
                Count_list = [0]*k
            #print(Agent_list[0].Q_values)
            for i in range(N):
                #For each agent
                #Get the index of the maximum q_value
                index = np.argmax(Agent_list[i].Q_values)
                Decision[index] = Decision[index] + 1 
            count = count+1
        return(Agent_list[0].G,Decision)
        
    
    

    


    
    

In [3]:
import itertools
trials = 200
N = 50
k = 6
b = 4 
score,decision = run(N,b,k,trials,reward_type = 0)


# Generate all combinations of num_elements that sum to target_sum
result = []
    
    # Use combinations_with_replacement to generate numbers that sum to target_sum
for combination in itertools.combinations_with_replacement(range(N+1), k):
        if sum(combination) == N:
            result.append(combination)

combinations = result
# Print the first 5 results to check
for comb in combinations[:5]:
    print(comb)

(0, 0, 0, 0, 0, 50)
(0, 0, 0, 0, 1, 49)
(0, 0, 0, 0, 2, 48)
(0, 0, 0, 0, 3, 47)
(0, 0, 0, 0, 4, 46)


In [4]:
#Individual Reward  
for comb in combinations:
    decision = np.array(comb)
    Nash = True
    for agent_no,total in enumerate(decision):
        Score = total*np.exp(-total/b)
        #print(decision)
        current_total = total
        #print(decision_new)
        #print(Score)
        #print(agent_no)
        if total == 0:
            continue
        decision[agent_no] = decision[agent_no] -1
        for i in range(len(decision)):
            #decision_new[i] = decision[i]+1
            #Calc reward
            New_decision_Total = decision[i]+1
            Reward = New_decision_Total*np.exp(-New_decision_Total/b)
            
            if Reward>Score:
                Nash = False
                #print(agent_no)
                #print(i)
                #print(New_decision_Total)
                #print(Score)
                #print(Reward)
        decision[agent_no] = decision[agent_no] +1
    if Nash==True:
        print(decision)
    


[ 0 10 10 10 10 10]
[8 8 8 8 9 9]


In [56]:
#Global Reward 
decision_new = np.zeros(k) 
for comb in combinations:
    decision = np.array(comb)
    Nash = True
    Score = 0
    #Do global instead of individual
    for agent,total_count in enumerate(decision):
        Score = total_count*np.exp(-total_count/b) +Score
    for agent_no,total in enumerate(decision):
        #Score = total*np.exp(-total/b)
        #print(decision)
        current_total = total
        #print(decision_new)
        #print(Score)
        #print(agent_no)
        if total == 0:
            continue
            
        decision[agent_no] = decision[agent_no] -1
        for i in range(len(decision)):
            decision_new = decision.copy()
            #Calc reward
            Total_count = decision[i].copy()
            decision_new[i] = Total_count+1
            #Reward = New_decision_Total*np.exp(-New_decision_Total/b)
            Reward = 0
            #Now calculate global reward
            for agent_no_rew,total_count_2 in enumerate(decision_new):
                Reward = total_count_2*np.exp(-total_count_2/b) +Reward
                
            #print(Reward)
            #print(decision)
            #print(Score)
            #print(decision_new)
            if Reward>Score:
                #print(agent_no)
                #print(comb)
                Nash = False
                #print(agent_no)
                #print(i)
                #print(decision)
            
                
            
        decision[agent_no] = decision[agent_no] +1
        #print(Score)
    if Nash==True:
        print(decision)
        
        pass
        

[ 4  4  4  4  4 30]
